In [1]:
# !pip install psycopg2
import psycopg2
from psycopg2.extensions import register_adapter
from psycopg2.extras import Json

register_adapter(dict, Json)

conn = psycopg2.connect("dbname=collegepdf user=postgres")

cur = conn.cursor()

sql_1 = "DROP TABLE IF EXISTS textract;"
sql_2 = r"""
CREATE TABLE textract (
  id SERIAL PRIMARY KEY,
  filepath VARCHAR(255) NOT NULL,
  filename VARCHAR(255) NOT NULL,
  college VARCHAR (255) NOT NULL,
  pdfminer TEXT,
  pdfminer_detailed JSON[],
  pypdf2 TEXT
);
""".replace('\n','')

for sql in [sql_1,sql_2]:
    cur.execute(sql)
    
sql_3 = """
INSERT INTO textract 
  (filepath, filename, college, pdfminer, pdfminer_detailed, pypdf2) 
  VALUES (%(filepath)s, %(filename)s, %(college)s, %(pdfminer)s, %(pdfminer_detailed)s::json[], %(pypdf2)s)
""".replace('\n','')

In [ ]:
import os

from tqdm.notebook import trange, tqdm

from utils.pdf_processing import get_all_data

files = [directory + '/' + file 
         for directory,_,files in list(os.walk('data')) 
         for file in files 
         if '.pdf' in file]

for f in tqdm(files):
    try:
        cur.execute(sql_3, get_all_data(f, 10))
    except Exception as e:
        print(f, e)

In [ ]:
cur.execute('select * from textract')
cur.fetchall()